In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
class BlogState(TypedDict):
    title: str
    outline : str
    content: str
    score: int

In [7]:
model = ChatOpenAI()


def create_outline(state: BlogState) -> BlogState:
    title = state["title"]
    prompt = f"Create an outline for a blog post with the title: {title}"
    outline = model.invoke(prompt).content
    state["outline"] = outline
    return state

def create_blog(state: BlogState) -> BlogState:
    title = state["title"]
    outline = state["outline"]
    prompt = f"Write a blog post with the title: {title} and the following outline: {outline}"
    content = model.invoke(prompt).content
    state["content"] = content
    return state    

def evaluate_blog(state: BlogState) -> BlogState:
    content = state['content']
    outline = state['outline']
    prompt = f"based on this outline: {outline} rate my blog post: {content} on a scale of 1 to 10"
    score = model.invoke(prompt).content
    state['score'] = score
    return state

In [8]:
graph = StateGraph(BlogState)


#node 1: generate a blog title
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)
graph.add_node("evaluate_blog", evaluate_blog)

# edges
graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", "evaluate_blog")
graph.add_edge("evaluate_blog", END)


#compile
workflow = graph.compile()

In [9]:
initial_state = {"title": "The Benefits of Using LangGraph for Workflow Management"}
final_state = workflow.invoke(initial_state)
print(final_state)

{'title': 'The Benefits of Using LangGraph for Workflow Management', 'outline': 'I. Introduction\n    A. Brief overview of workflow management\n    B. Introduction to LangGraph\n\nII. Streamlined Communication\n    A. How LangGraph helps facilitate communication among team members\n    B. The importance of clear communication in workflow management\n\nIII. Increased Efficiency\n    A. The features of LangGraph that promote efficiency\n    B. Case studies or examples of companies seeing increased efficiency with LangGraph\n\nIV. Resource Allocation\n    A. How LangGraph helps with resource allocation and task prioritization\n    B. The impact of effective resource allocation on workflow management\n\nV. Time Savings\n    A. How using LangGraph can save time for team members and managers\n    B. The long-term benefits of time savings in workflow management\n\nVI. Enhanced Collaboration\n    A. The collaborative features of LangGraph\n    B. The impact of collaboration on overall workflow

In [10]:
print(final_state['outline'])
print(final_state['content'])
print(final_state['score'])

I. Introduction
    A. Brief overview of workflow management
    B. Introduction to LangGraph

II. Streamlined Communication
    A. How LangGraph helps facilitate communication among team members
    B. The importance of clear communication in workflow management

III. Increased Efficiency
    A. The features of LangGraph that promote efficiency
    B. Case studies or examples of companies seeing increased efficiency with LangGraph

IV. Resource Allocation
    A. How LangGraph helps with resource allocation and task prioritization
    B. The impact of effective resource allocation on workflow management

V. Time Savings
    A. How using LangGraph can save time for team members and managers
    B. The long-term benefits of time savings in workflow management

VI. Enhanced Collaboration
    A. The collaborative features of LangGraph
    B. The impact of collaboration on overall workflow management success

VII. Flexibility
    A. How LangGraph can adapt to different workflow management n

In [11]:
print(final_state['score'])

I would rate this blog post a 9. It is well-organized, informative, and effectively highlights the key benefits of using LangGraph for workflow management. The outline is clear and easy to follow, and the content effectively conveys the value of the tool. The only potential improvement could be to include some specific statistics or data to support the claims made about the benefits of using LangGraph.
